In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf
import lmbspecialops as sops

In [3]:
import sys
sys.path.insert(0,'/misc/lmbraid19/thomasa/catkin_ws/src/reinforced_visual_slam/networks/depth_fusion')

In [4]:
from net.my_networks import *
from net.my_losses import *

In [5]:

feature_names = [
'rgb',
'sparseInverseDepth',
'sparseInverseDepthVariance'
]

FLAGS = {
  'multi_gpu': True,
  'batch_size':12,
  'prefetch_buffer_size': 60,
  'num_parallel_calls': 24,
  'num_epochs':50,
  'learning_rate':0.0005,
  'data_format': "channels_first"
}

In [6]:
def dataset_shuffler(training_file):
  filename_records = tf.data.TextLineDataset(training_file)
  dataset = filename_records.shuffle(buffer_size=50000)
  return dataset

def parse_load_augment(filename_line_):
  filenames = tf.decode_csv(filename_line_, [[''], [''], [''], ['']])
  print(filenames.output_shapes)
  indices = tf.data.Dataset.from_tensors([0, 1, 2, 3])
  print(indices.output_shapes)
  image_files = tf.data.Dataset.zip((filenames, indices))
  images = images_files.map(map_func=load_image, num_parallel_calls=4)
  # add augmentation step here
  depth_gt = images.range(0, 1)
  train_images = images.range(1, 4)
  d = dict(zip(feature_names, train_images)), depth_gt
  return d

def invert_clean(x, threshold=0.01):
    mask =  tf.logical_and(tf.is_finite(x), x>threshold)
    x_clean = tf.where(mask, tf.reciprocal(x), tf.zeros_like(x))
    return x_clean

def load_image(filename, index, data_format=FLAGS['data_format'], width=320, height=240, 
  resize=True, basepath = "/misc/lmbraid19/thomasa/datasets/LSDDepthTraining/train/"):
  image_string = tf.read_file(basepath + filename)
  if index == 0:
    # current image is depth groundtruth
    image_decoded = tf.image.decode_png(image_string, dtype=tf.uint16)
    image_decoded = tf.reshape(image_decoded, [480, 640, 1])
    image_decoded = tf.cast(image_decoded, tf.float32)
    image_decoded = tf.scalar_mul(0.0002, image_decoded)
    # converting (and clean) depth to idepth and removing NaN's and Inf's
    image_decoded = invert_clean(image_decoded)

  elif index == 1:
    # current image is rgb
    image_decoded = tf.image.decode_png(image_string)
    image_decoded = tf.reshape(image_decoded, [480, 640, 3])
    image_decoded = tf.cast(image_decoded, tf.float32)
    # converting rgb to [0,1] from [0,255]
    image_decoded = tf.scalar_mul(0.003926, image_decoded)

  else:
    # for both the sparse depth and variance
    image_decoded = tf.decode_raw(image_string, tf.half)
    image_decoded = tf.reshape(image_decoded, [480, 640, 1])
    image_decoded = tf.cast(image_decoded, tf.float32)
    image_decoded = replace_nonfinite(image_decoded)

  if resize == True:
    image_decoded = tf.image.resize_images(image_decoded, [height, width], align_corners=True)
  if data_format == 'channels_first':
    image_decoded = tf.transpose(image_decoded,[2,0,1])
  return image

In [7]:
training_file = "/misc/lmbraid19/thomasa/datasets/LSDDepthTraining/train/rgbd_lsdDepth_train.txt"
dataset = dataset_shuffler(training_file)
dataset

<ShuffleDataset shapes: (), types: tf.string>

In [8]:
def parse_n_load(filename_line, data_format=FLAGS['data_format'], width=320, height=240):
  filenames = tf.decode_csv(filename_line, [[''], [''], [''], ['']])
  images = []
  basepath = "/misc/lmbraid19/thomasa/datasets/LSDDepthTraining/train/"
  for cnt in range(4):
      image_string = tf.read_file(basepath + filenames[cnt])
      if cnt < 2:
        if cnt == 0:
          image_decoded = tf.image.decode_png(image_string, dtype=tf.uint16)
          image_decoded = tf.reshape(image_decoded, [480, 640, 1])
        else:
          image_decoded = tf.image.decode_png(image_string)
          image_decoded = tf.reshape(image_decoded, [480, 640, 3])
        image_decoded = tf.cast(image_decoded, tf.float32)
      else:
        image_decoded = tf.decode_raw(image_string, tf.half)
        image_decoded = tf.reshape(image_decoded, [480, 640, 1])
        image_decoded = tf.cast(image_decoded, tf.float32)
        image_decoded = replace_nonfinite(image_decoded)
      image_decoded = tf.image.resize_images(image_decoded, [height, width], 
        align_corners=True)
      if data_format == 'channels_first':
        image_decoded = tf.transpose(image_decoded,[2,0,1])
      images.append(image_decoded)
          
  depth_gt = tf.scalar_mul(0.0002, images[0])
  # converting depth to idepth and removing NaN's and Inf's
  #idepth_gt = tf.reciprocal(depth_gt)
  #idepth_gt_clean = replace_nonfinite(idepth_gt)
  idepth_gt_clean = invert_clean(depth_gt)
  del images[0]
  # converting rgb to [0,1] from [0,255]
  images[0] = tf.scalar_mul(0.003926, images[0])
  #d = dict(zip(feature_names, images)), idepth_gt_clean
  d = dict(zip(feature_names, images)), idepth_gt_clean
  return d

def dataset_shuffler(training_file):
  filename_records = tf.data.TextLineDataset(training_file)
  dataset = filename_records.shuffle(buffer_size=50000)
  return dataset

def input_fn(dataset, shuffle=False):
  dataset = dataset.map(map_func=parse_n_load, num_parallel_calls=FLAGS['num_parallel_calls'])
  #dataset.apply(tf.contrib.data.map_and_batch(map_func=parse_n_load, num_parallel_calls=FLAGS.num_parallel_calls,
   #                                           batch_size=FLAGS.batch_size))
  dataset = dataset.repeat(FLAGS['num_epochs'])
  if shuffle:
    dataset = dataset.shuffle(buffer_size=10000)
  dataset = dataset.batch(batch_size=FLAGS['batch_size'])
  dataset = dataset.prefetch(buffer_size=FLAGS['prefetch_buffer_size'])
  iterator = dataset.make_one_shot_iterator()
  features, label = iterator.get_next()
  return features, label

def train_input_fn():
  training_file = "/misc/lmbraid19/thomasa/datasets/LSDDepthTraining/train/rgbd_lsdDepth_train.txt"
  #basepath = os.path.dirname(training_file)
  dataset = dataset_shuffler(training_file)
  #dataset = tf.data.TextLineDataset(training_file)
  features, label = input_fn(dataset, shuffle=False)
  return features, label

def replace_nonfinite(x):
    mask = tf.is_finite(x)
    x_clean = tf.where(mask, x, tf.zeros_like(x))
    return x_clean

def validate_batch_size_for_multi_gpu(batch_size):
  """For multi-gpu, batch-size must be a multiple of the number of GPUs.
  Note that this should eventually be handled by replicate_model_fn
  directly. Multi-GPU support is currently experimental, however,
  so doing the work here until that feature is in place.
  Args:
    batch_size: the number of examples processed in each training batch.
  Raises:
    ValueError: if no GPUs are found, or selected batch_size is invalid.
  """
  from tensorflow.python.client import device_lib  # pylint: disable=g-import-not-at-top

  local_device_protos = device_lib.list_local_devices()
  num_gpus = sum([1 for d in local_device_protos if d.device_type == 'GPU'])
  if not num_gpus:
    raise ValueError('Multi-GPU mode was specified, but no GPUs '
                     'were found. To use CPU, run without --multi_gpu.')

  remainder = batch_size % num_gpus
  if remainder:
    err = ('When running with multiple GPUs, batch size '
           'must be a multiple of the number of available GPUs. '
           'Found {} GPUs with a batch size of {}; try --batch_size={} instead.'
          ).format(num_gpus, batch_size, batch_size - remainder)
    raise ValueError(err)

In [9]:
def model_fn(features, labels, mode, params):
  """The model_fn argument for creating an Estimator."""
  rgb = features['rgb']
  sparseIdepth = features['sparseInverseDepth']
  sparseIdepthVar = features['sparseInverseDepthVariance']
  inputs = [rgb, sparseIdepth, sparseIdepthVar]

  # summaries for images
  if params.get('data_format') == "channels_first":
    rgb_nhwc = tf.transpose(rgb, [0,2,3,1])
    sparseIdepth_nhwc = tf.transpose(sparseIdepth, [0,2,3,1])
    sparseIdepthVar_nhwc = tf.transpose(sparseIdepthVar, [0,2,3,1])
  else:
    rgb_nhwc = rgb
    sparseIdepth_nhwc = sparseIdepth
    sparseIdepthVar_nhwc = sparseIdepthVar
  tf.summary.image('rgb', rgb_nhwc, max_outputs=1)
  tf.summary.image('sparseIdepth', sparseIdepth_nhwc, max_outputs=1)
  tf.summary.image('sparseIdepthVar', sparseIdepthVar_nhwc, max_outputs=1)

  model = NetworkV02()

  if mode == tf.estimator.ModeKeys.PREDICT:
    depth = model(inputs, params.get('data_format'))
    predictions = {
        'depth': depth,
    }
    return tf.estimator.EstimatorSpec(mode=tf.estimator.ModeKeys.PREDICT, 
                                  predictions=predictions, 
                                  export_outputs={'depth': tf.estimator.export.PredictOutput(predictions)})

  if mode == tf.estimator.ModeKeys.TRAIN:
    learning_rate_base = 0.001
    learning_rate = tf.train.exponential_decay(learning_rate_base, tf.train.get_or_create_global_step(), 
      decay_rate=0.8, decay_steps=100000, staircase=False)
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

    # If we are running multi-GPU, we need to wrap the optimizer.
    if params.get('multi_gpu'):
      optimizer = tf.contrib.estimator.TowerOptimizer(optimizer)
    
    depth = model(inputs, data_format=params.get('data_format'))
    loss = pointwise_l1_loss(inp=depth, gt=labels, data_format=params.get('data_format'))
    if params.get('data_format') == "channels_first":
      depth_nhwc = tf.transpose(depth, [0,2,3,1])
      labels_nhwc = tf.transpose(labels, [0,2,3,1])
    else:
      depth_nhwc = depth
      labels_nhwc = labels
    tf.summary.image('depthPredicted', depth_nhwc, max_outputs=1)
    tf.summary.image('depthGt', labels_nhwc, max_outputs=1)  

    # Save scalars to Tensorboard output.
    tf.summary.scalar('train_loss', loss)
    tf.summary.scalar('learning_rate', learning_rate)

    return tf.estimator.EstimatorSpec(mode=tf.estimator.ModeKeys.TRAIN, loss=loss, 
      train_op=optimizer.minimize(loss, tf.train.get_or_create_global_step()))

  if mode == tf.estimator.ModeKeys.EVAL:
    depth = model(inputs, params.get('data_format'))
    loss = pointwise_l2_loss(inp=labels, gt=depth, data_format=params.get('data_format'))
    return tf.estimator.EstimatorSpec(mode=tf.estimator.ModeKeys.EVAL,
                                      loss=loss)


In [ ]:
def run_trainer(model_dir_name, training_steps):
  """Run training and eval loop for lsd depth fusion network.
  """
  model_function = model_fn
  hooks = [tf_debug.LocalCLIDebugHook()]
  #hooks = [tf_debug.TensorBoardDebugHook("localhost:6064")] (tf 1.5+)

  if FLAGS['multi_gpu']:
    validate_batch_size_for_multi_gpu(FLAGS['batch_size'])

    # There are two steps required if using multi-GPU: (1) wrap the model_fn,
    # and (2) wrap the optimizer. The first happens here, and (2) happens
    # in the model_fn itself when the optimizer is defined.
    model_function = tf.contrib.estimator.replicate_model_fn(
        model_fn, loss_reduction=tf.losses.Reduction.MEAN)

  data_format = 'channels_first'
  model_base_dir = "/misc/lmbraid19/thomasa/catkin_ws/src/reinforced_visual_slam/networks/depth_fusion/training"
  model_dir = os.path.join(model_base_dir, model_dir_name)

  lsd_depth_fuser = tf.estimator.Estimator(
    model_fn=model_function,
    model_dir=model_dir,
    params={
    'data_format': data_format,
    'multi_gpu': FLAGS['multi_gpu']
    })

  # Train and evaluate model.
  #for _ in range(FLAGS['num_epochs'] // flags_obj.epochs_between_evals):
  lsd_depth_fuser.train(input_fn=train_input_fn, steps=training_steps)

In [10]:
features, labels = train_input_fn()
rgb = features['rgb']
sparseIdepth = features['sparseInverseDepth']
sparseIdepthVar = features['sparseInverseDepthVariance']
inputs = [rgb, sparseIdepth, sparseIdepthVar]
#tf.summary.image('rgb', rgb, max_outputs=6)
#tf.summary.image('sparseIdepth', sparseIdepth, max_outputs=6)
#tf.summary.image('sparseIdepthVar', sparseIdepthVar, max_outputs=6)
inputs = [rgb, sparseIdepth, sparseIdepthVar]


In [11]:
sess = tf.Session()
inputs_pr = sess.run(inputs)

In [12]:
labels_pr = sess.run(labels)

In [13]:
labels_pr[11].shape

(1, 240, 320)

In [22]:
np.max(labels_pr[11])


97.374504

In [ ]:
image_string = tf.read_file('/misc/lmbraid19/thomasa/datasets/LSDDepthTraining/train/rgbd_dataset_freiburg1_360/99_1305031803.381500_42_depthGT.png')

In [ ]:
sess2 = tf.Session()

In [ ]:
depth_string = sess2.run(image_string)

In [ ]:
image_decoded = tf.image.decode_png(image_string, dtype=tf.uint16)


In [ ]:
image_decoded = tf.reshape(image_decoded, [480, 640, 1])
image_decoded = tf.cast(image_decoded, tf.float32)

In [ ]:
image_reshaped = tf.image.resize_images(image_decoded, [240, 320], align_corners=True)

In [ ]:
depth_gt = tf.transpose(image_reshaped,[2,0,1])
depth_gt = tf.scalar_mul(0.0002, depth_gt)
depths = [image_decoded, image_reshaped, depth_gt]

In [ ]:
idepth_gt = tf.reciprocal(depth_gt)
idepth_gt_clean = replace_nonfinite(idepth_gt)

In [ ]:
idepth_gt_clean_ = sess2.run(idepth_gt_clean)

In [ ]:
depths_ = sess2.run(depths)

In [ ]:
image_decoded = tf.reshape(image_decoded, [480, 640, 1])
image_decoded = tf.cast(image_decoded, tf.float32)
image_decoded = tf.transpose(image_decoded,[2,0,1])
depth_gt = tf.scalar_mul(0.0002, image_decoded)

In [ ]:
depth_decoded = sess2.run(image_decoded)
depth_gt_ = sess2.run(depth_gt)

In [ ]:
print(idepth_gt_clean_.shape)
print(idepth_gt_clean_.dtype)
print(np.max(idepth_gt_clean_))
print(np.min(idepth_gt_clean_))

In [ ]:
idepth_ = np.transpose(idepth_gt_clean_, [1, 2 ,0])

In [ ]:
idepth_.shape

In [ ]:
import PIL
import matplotlib.pyplot as plt
from matplotlib import colors

In [ ]:
plt.imshow(idepth_[:,:,0], cmap='hot')
plt.colorbar()



In [ ]:
model = NetworkV02()
depth = model(inputs, data_format='channels_first')

In [ ]:
loss1 = pointwise_l1_loss(inp=depth, gt=labels, data_format='channels_first')
loss2 = pointwise_l2_loss(inp=depth, gt=labels, data_format='channels_first')
loss = [loss1, loss2]

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
loss_pr = sess.run(loss)

In [ ]:
diff = depth - labels
sop_diff = sops.replace_nonfinite(diff)
diffs = [diff, sop_diff]

In [ ]:
sop_pow = sop_diff**2
diff_pow = diff**2
pows = [diff_pow, sop_pow]

In [ ]:
pow_sop = tf.pow(sop_diff, 2)
pow_diff = tf.pow(diff, 2)
pows_tf_ = [pow_diff, pow_sop]

In [ ]:
pows_tf_pr = sess.run(pows_tf_)

In [ ]:
np.isinf(pows_tf_pr[1]).any()


In [ ]:
np.isnan(diffs_pr[1]).any()

In [ ]:
labels_pr = sess.run(labels)

In [ ]:
depth_pr = sess.run(depth)

In [ ]:
labels_pr = sess3.run(labels)

In [ ]:
print(np.max(inputs_pr[2]))

In [ ]:
print(np.max(labels_pr) * 5000)

In [ ]:
print(np.nanmax(1/depth_pr))
print(np.min(depth_pr))

In [ ]:
depth_pr**2

In [ ]:
model = NetworkV01()
depth = model(inputs)

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
depth_pr = sess.run(depth)

In [ ]:
depth_pr

In [ ]:
writer = tf.summary.FileWriter("/misc/lmbraid19/thomasa/catkin_ws/src/reinforced_visual_slam/networks/depth_fusion/training/02")
writer.add_graph(sess.graph)

In [ ]:
#model = SimpleNetwork()
#model.compile(optimizer= tf.keras.optimizers.Adam(0.001), loss=pointwise_l2_loss)

In [ ]:
def run_trainer(flags_obj):
  """Run MNIST training and eval loop.
  Args:
    flags_obj: An object containing parsed flag values.
  """

  model_function = model_fn

  if flags_obj.multi_gpu:
    validate_batch_size_for_multi_gpu(flags_obj.batch_size)

    # There are two steps required if using multi-GPU: (1) wrap the model_fn,
    # and (2) wrap the optimizer. The first happens here, and (2) happens
    # in the model_fn itself when the optimizer is defined.
    model_function = tf.contrib.estimator.replicate_model_fn(
        model_fn, loss_reduction=tf.losses.Reduction.MEAN)

  data_format = flags_obj.data_format
  if data_format is None:
    data_format = ('channels_first'
                   if tf.test.is_built_with_cuda() else 'channels_last')

  lsd_depth_fuser = tf.estimator.Estimator(
      model_fn=model_function,
      model_dir=flags_obj.model_dir,
      params={
      'data_format': data_format,
      'multi_gpu': flags_obj.multi_gpu
      })

  # Train and evaluate model.
  for _ in range(10):
    lsd_depth_fuser.train(input_fn=train_input_fn(dataset.range(train_ind)))
    eval_results = lsd_depth_fuser.evaluate(input_fn=eval_input_fn)
    print('\nEvaluation results:\n\t%s\n' % eval_results)

    if model_helpers.past_stop_threshold(flags_obj.stop_threshold,
                                         eval_results['loss']):
      break

In [ ]:
for _ in range(10):
    lsd_depth_fuser.train(input_fn=train_input_fn(dataset.range(train_ind)))
    eval_results = lsd_depth_fuser.evaluate(input_fn=train_input_fn(dataset.range(train_ind, train_ind+100000)))
    print('\nEvaluation results:\n\t%s\n' % eval_results)

In [ ]:
"""
To walk throught subdirectories of the training directory and 
make a single file containing relative path to each of the training samples.
Each line in file would be:
    depth_gt,rgb,sparseDepth,sparseDepthVar
"""
import os
with open("/misc/lmbraid19/thomasa/datasets/rgbd_lsdDepth_train.txt", "w+") as train_file:
    cnt = 0
    for root, directories, filenames in os.walk('/misc/lmbraid19/thomasa/datasets/LSDDepthTraining/'):
        for files in sorted(filenames):
            cnt += 1
            if cnt < 4:
                sep = ","
            else:
                sep = "\r\n"
                cnt = 0
            #print(files)
            #print(os.path.basename(os.path.normpath(root)))
            train_file.write( os.path.basename(os.path.normpath(root)) + "/" + files + sep)

In [ ]:
import os
folder = "/misc/lmbraid19/thomasa/datasets"
folder_parent = os.path.dirname(folder)
print(folder_parent)
os.path.join(folder_parent, "lsdDepth.txt")


In [ ]:
st = ("enthanu "
     "nannayikoode")
print(st)

In [ ]:
feature_names = [
'rgb',
'sparseInverseDepth',
'sparseInverseDepthVariance'
]
len(feature_names)

In [ ]:
class SimpleNetwork( object ):

  def __init__(self):

    self.FLAGS = {
    'batch':8,
    }

    # starting conv rgb
    #self.convrgb = tf.keras.layers.Conv2D(filters=16, kernel_size=5, activation=tf.nn.relu, input_shape=(self.FLAGS['batch'],640,480,3), padding="same")
    self.convrgb = tf.keras.layers.Conv2D(filters=16, kernel_size=5, activation=tf.nn.relu, padding="same")

    # starting conv sparse
    #self.convsparse = tf.keras.layers.Conv2D(filters=16, kernel_size=5, activation=tf.nn.relu, input_shape=(self.FLAGS['batch'],640,480,1), padding="same")
    self.convsparse = tf.keras.layers.Conv2D(filters=16, kernel_size=5, activation=tf.nn.relu, padding="same")

    # encoder network:
    self.conv1 = tf.keras.layers.Conv2D(filters=32, kernel_size=5, activation=tf.nn.relu, padding="same")
    self.pool1 = tf.keras.layers.MaxPool2D(pool_size=2, strides=2)

    self.conv2 = tf.keras.layers.Conv2D(filters=63, kernel_size=5, activation=tf.nn.relu, padding="same")
    self.pool2 = tf.keras.layers.MaxPool2D(pool_size=2, strides=2)

    self.conv3 = tf.keras.layers.Conv2D(filters=128, kernel_size=5, activation=tf.nn.relu, padding="same")
    self.pool3 = tf.keras.layers.MaxPool2D(pool_size=2, strides=2)

    # decoder network:

    self.deconv3 = tf.keras.layers.Conv2D(filters=128, kernel_size=5, activation=tf.nn.relu, padding="same")
    self.upsample3 = tf.keras.layers.UpSampling2D(size=2)

    self.deconv2 = tf.keras.layers.Conv2D(filters=64, kernel_size=5, activation=tf.nn.relu, padding="same")
    self.upsample2 = tf.keras.layers.UpSampling2D(size=2)

    self.deconv1 = tf.keras.layers.Conv2D(filters=32, kernel_size=5, activation=tf.nn.relu, padding="same")
    self.upsample1 = tf.keras.layers.UpSampling2D(size=2)

    self.dropout = tf.keras.layers.Dropout(0.5)
    self.deconv = tf.keras.layers.Conv2D(filters=1, kernel_size=1, activation=tf.nn.relu, padding="same")

  def encoder_decoder(self, inputs, training=False):
  	x = self.conv1(inputs)
  	x = self.pool1(x)
  	x = self.conv2(x)
  	x = self.pool2(x)
  	x = self.conv3(x)
  	x = self.pool3(x)
  	x = self.deconv3(x)
  	x = self.upsample3(x)
  	x = self.deconv2(x)
  	x = self.upsample2(x)
  	x = self.deconv1(x)
  	x = self.upsample1(x)
  	return x

  def rgb_net(self, rgb):
  	x = self.convrgb(rgb)
  	return x

  def sparse_net(self, sparse_image):
  	x = self.convsparse(sparse_image)
  	return x

  def __call__(self, inputs, training=False):
    with tf.variable_scope('keras_model'):
      #rgb = inputs['rgb']
      #sparse_depth = inputs['sparseInverseDepth']
      #sparse_depth_var = inputs['sparseInverseDepthVariance']
      rgb, sparse_depth, sparse_depth_var = inputs
      x1 = self.rgb_net(rgb)
      x2 = self.sparse_net(sparse_depth)
      x3 = self.sparse_net(sparse_depth_var)
      x = tf.keras.layers.concatenate( [x1,x2,x3])
      x = self.encoder_decoder(x)
      if training:
        x = self.dropout(x)
      return self.deconv(x)

In [ ]:
from __future__ import division

def dataset_len(dataset_loc):
    with open(dataset_loc) as f:
        for i, l in enumerate(f):
            pass
    return i + 1

In [ ]:
training_file = "/misc/lmbraid19/thomasa/datasets/LSDDepthTraining/train/rgbd_lsdDepth_train.txt"
data_len = dataset_len(training_file)

In [ ]:
batch_size = 12
steps_per_epoch = data_len // batch_size

In [ ]:
print(data_len)
print(batch_size)
print(steps_per_epoch)

In [ ]:
num_epoch = 50
print("num_steps = ", steps_per_epoch*num_epoch)

In [ ]:
idepth_prediction_file = "/home/thomasa/Downloads/prediction.png"
idepth_gt_file = "/home/thomasa/Downloads/gt2.png"

In [ ]:
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors


In [ ]:
#idepth_prediction = mpimg.imread(idepth_prediction_file)
idepth_gt = mpimg.imread(idepth_gt_file)

In [ ]:
plt.hist(idepth_gt)

In [ ]:
fig, axs = plt.subplots(1, 2)
norm = colors.Normalize(vmin=0, vmax=0.07)
cmap = 'hot'
images = []
images.append(axs[0].imshow(idepth_prediction, cmap=cmap))
images.append(axs[1].imshow(idepth_gt, cmap=cmap))
for im in images:
    im.set_norm(norm)
fig.colorbar(images[0], ax=axs, orientation='horizontal', fraction=.1)